In [2]:
!pip3 install pydicom -q

     |████████████████████████████████| 1.9MB 5.7MB/s 


In [3]:
#Run once per session
!pip install fastai -q --upgrade

     |████████████████████████████████| 194kB 4.3MB/s 
     |████████████████████████████████| 61kB 4.4MB/s 


In [ ]:
# !pip install voila
# !jupyter serverextension enable --sys-prefix voila

In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *

import numpy as np
from scipy.ndimage.interpolation import zoom
import pydicom as py
from io import BytesIO
import os, os.path as osp
import cv2

In [2]:
def get_x(r): return data_folder + r['filename'] + '.jpg'
def get_y(r): return r['labels'].split(',')
learn = load_learner('Downloads/export.pkl')

In [20]:
def convert(dicom_file):
    def window(img, WL=50, WW=350):
        upper, lower = WL+WW//2, WL-WW//2
        X = np.clip(img.copy(), lower, upper)
        X = X - np.min(X)
        X = X / np.max(X)
        X = (X*255.0).astype('uint8')
        
        return X
        
    if "PixelData" not in dicom_file:
        image = np.zeros(imsize)
    else:
        image = dicom_file.pixel_array.copy()

    scale_slope = 1.0
    scale_intercept = 0.0
    if "RescaleSlope" in dicom_file:
        scale_slope = float(dicom_file.RescaleSlope)
    if "RescaleIntercept" in dicom_file:
        scale_intercept = float(dicom_file.RescaleIntercept)
    image = image + scale_intercept
    image = image * scale_slope
    
    try:
        # Windows
        image_lung = np.expand_dims(window(image, WL=-600, WW=1500), axis=2)
        image_abdomen = np.expand_dims(window(image, WL=40, WW=350), axis=2)
        image_bone = np.expand_dims(window(image, WL=480, WW=2500), axis=2)
        
        image = np.concatenate([image_lung, image_abdomen, image_bone],axis=2)
        
        rat = 64. / np.max(image.shape[1:])
        image_scaled = zoom(image, [rat,rat,1.], prefilter=False, order=1)
        return image_scaled
    except Exception as e:
        print(e)
        return image

In [91]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [94]:
def on_data_change(change):
  lbl_pred.value = ''
  is_jpg = True
  image_data = btn_upload.data[-1]

  try:
    img = PILImage.create(image_data)
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    #resize
    img = img.resize((64,64))
    img = TensorImage(image2tensor(img))
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
  except:
    is_jpg = False
  
  if is_jpg == False:
    ds = py.dcmread(BytesIO(image_data))
    img = convert(ds)
    tmpdir = tempfile.TemporaryDirectory()
    tmp_jpg_for_prediction = osp.join(tmpdir.name, 'tmp.jpg')
    cv2.imwrite(tmp_jpg_for_prediction, img)
    out_pl.clear_output()
    with out_pl: display(Image.fromarray(img))
    pred,pred_idx,probs = learn.predict(tmp_jpg_for_prediction)
  
  lbl_pred.value = f'Prediction: {pred}'

btn_upload.observe(on_data_change, names=['data'])

In [93]:
display(VBox([widgets.Label('Select a CT Image'), btn_upload, out_pl, lbl_pred]))

Not a jpg
